In [43]:
import cv2
import numpy as np

import torch
import torchvision.transforms as transforms
from PIL import Image as PILImage, ImageDraw
# import PIL
import matplotlib.pyplot as plt
import argparse
import os
from IPython.display import clear_output, display, Image
from facenet_pytorch import MTCNN, InceptionResnetV1

from torch import nn, optim, as_tensor
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
from models.inception_resnet_v1 import InceptionResnetV1

IPython.core.display.Image

In [56]:
cuda_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Running on device: {cuda_device}')

Running on device: cuda:0


In [57]:
face_detect_model = MTCNN(keep_all=True, device=cuda_device)

### Jerico

In [11]:
%pwd

'/mids-w251-project/Final Project/mids-w251-project/face/m1'

In [58]:
#Perform transforms and normalization to prepare images for training on ResNet
data_transforms = {
    'train': transforms.Compose([
        #Note: Our images must be 160x160 for model (Training & Val). 
        transforms.Resize(size = (160,160)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(size = (160,160)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
data_dir = 'data/train_val'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                              batch_size=8, 
                                             shuffle=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train','val']}
class_names = image_datasets['train'].classes
class_names

['Diana_Chacon', 'Jerico_Johns', 'Josh_Jonte', 'Sudhrity_Mondal']

In [78]:
#Jerico part
#model_ft = InceptionResnetV1(pretrained='vggface2', classify=False)

#model_ft.load_state_dict(torch.load('models/face_recognition_transf.pt'), strict = False)
model_ft = torch.load('models/face_recognition_transf.pt')
model_ft.to(cuda_device)
model_ft.eval()

Sequential(
  (0): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (1): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (2): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (5): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3,

In [79]:
def get_name(face_img):
    with torch.no_grad(): 
        face_img = PILImage.fromarray(face_img)
        val_transform = data_transforms['val']
        face_img = val_transform(face_img).unsqueeze(0)
        face_img = face_img.to(cuda_device)
        output = model_ft(face_img)
        _, pred = torch.max(output, 1)
        return pred

def get_emotion(face_img):
    return 'happy'

In [85]:
video_capture = cv2.VideoCapture(0)
frame_count = 0

try:
    while(True):
        frame_count += 1
        
        clear_output(wait=True)
        _, frame = video_capture.read()
        faces, _ = face_detect_model.detect(frame)
        
        if faces is not None:
            for top, left, bottom, right  in faces:
                
                face_img = frame[0:300, 0:200]
                
                cv2.rectangle(frame, (top, left), (bottom, right), (0, 100, 0), 2)
                
                # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
                rgb_face = face_img[:, :, ::-1]
                face_name = get_name(rgb_face)
#                 face_emotion = get_emotion(face_img)
                font = cv2.FONT_HERSHEY_DUPLEX
                x, y, w, h = left+2, bottom-50, right-2, bottom-26
                #sub_img = frame[y:h, x:w]
                white_rect = np.ones(frame.shape, dtype=np.uint8) * 255
                res = cv2.addWeighted(frame, 0.8, white_rect, 0.1, 1.0)
                frame[y:h, x:w] = res

                scale = 0.05 # this value can be from 0 to 1 (0,1] to change the size of the text relative to the image
                fontScale = min(w,h)/(25/scale)

                #cv2.putText(frame, name, (left + 6, bottom - 29), font, 1.0, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(frame, face_name, (left + 6, bottom - 29), font, fontScale, (255, 255, 255), 1, cv2.LINE_AA)


                ret, img = cv2.imencode('.jpg', frame)
                img = Image(data=img)
                display(img)
finally:
    video_capture.release()
  
        
    

TypeError: slice indices must be integers or None or have an __index__ method